## This is a Jupyter notebook to set up and run a single ended dimer transition state searching calculation using the MACE-MP-0 interatomic potential and the Atomic Simulation Environment (ASE).

Steps 1-3 of creating an input supercell structure are covered in the notebook mace_ase_optimise_lifepo4.ipynb.

Sections that require user input for a specific system are shown as ################## User input ################# blocks.

### References:

#### ASE:
Paper: Larsen, A. H., Mortensen, J. J., Blomqvist, J., Castelli, I. E., Christensen, R., Dułak, M., ... & Jacobsen, K. W. (2017). The atomic simulation environment—a Python library for working with atoms. Journal of Physics: Condensed Matter, 29(27), 273002.

https://wiki.fysik.dtu.dk/ase/ase/io/io.html

https://wiki.fysik.dtu.dk/ase/ase/dimer.html

#### MACE:

Code: https://github.com/ACEsuit/mace-mp?tab=readme-ov-file

Paper: Batatia, I., Benner, P., Chiang, Y., Elena, A.M., Kovács, D.P., Riebesell, J., Advincula, X.R., Asta, M., Baldwin, W.J., Bernstein, N. and Bhowmik, A., 2023. A foundation model for atomistic materials chemistry. arXiv preprint arXiv:2401.00096.

#### Dimer:

Henkelman, G. and Jónsson, H., 1999. A dimer method for finding saddle points on high dimensional potential surfaces using only first derivatives. The Journal of chemical physics, 111(15), pp.7010-7022.

https://theory.cm.utexas.edu/vtsttools/dimer.html

## Step 0: Import some modules from ASE and MACE

In [1]:
import ase
from ase.optimize import LBFGS
from ase.io import read, write
from ase.filters import FrechetCellFilter
from ase.mep import DimerControl, MinModeAtoms, MinModeTranslate
from ase.build import make_supercell
from ase.io.trajectory import Trajectory
from ase import Atoms
from ase.constraints import FixAtoms
from mace.calculators import mace_mp
from ase.visualize import view
import numpy as np

import matplotlib.pyplot as plt

Read in the 1x2x2 supercell structure of LiFePO4 that we created previously.

In [2]:
fully_occupied_super=read('supercell.cif',format='cif')
view(fully_occupied_super) 

<Popen: returncode: None args: ['/Library/Frameworks/Python.framework/Versio...>

## Step 4: Create a single end member structure

Visualise the structure in ase gui with the view(fully_occupied_super) command (or another structure visualiser such as VESTA) and find the atom indices(list position) of two neighbouring Li sites along the b-axis of the cell. Bear in mind that ase uses 'python counting' so atoms 30 and 85 in ASE are atoms 31 and 86 in Vesta. Once we have selected the two neighbouring Li sites, we can delete them from the cell with 'del'.

In [3]:
################## User input #################
atom_1=30
atom_2=85
################## User input #################

In [4]:
pos_1=fully_occupied_super[atom_1].scaled_position
pos_2=fully_occupied_super[atom_2].scaled_position
min_cell_1=fully_occupied_super.copy()
del min_cell_1[[atom_1,atom_2]]

Now we need to create a new supercell (min_cell_1) with a Li atom in atom position 1 (pos_1) that we just identified. We do this by adding Li back into the structure we just created containing two Li vacancies. Once we have added a Li to the end of the file ([-1]), we optimise the atomic positions in the new supercell.

In [5]:
################## User input #################
insert_atom_type='Li'
################## User input #################

In [6]:
min_cell_1.append(insert_atom_type)
min_cell_1[-1].scaled_position=pos_1
min_cell_1.calc=mace_mp(default_dtype="float32")

bfgs=LBFGS(min_cell_1)
traj_opt = Trajectory('min_cell_1_opt.traj', 'w', min_cell_1)
bfgs.attach(traj_opt)
bfgs.run(fmax=0.01)

e0=min_cell_1.get_potential_energy()
write('min_cell_1.cif',min_cell_1,format='cif')
view(min_cell_1, viewer='x3d')

Using Materials Project MACE for MACECalculator with /Users/ieuanseymour/.cache/mace/5yyxdm76
Using float32 for MACECalculator, which is faster but less accurate. Recommended for MD. Use float64 for geometry optimization.
Default dtype float32 does not match model dtype float64, converting models to float32.
       Step     Time          Energy          fmax
LBFGS:    0 00:15:11     -759.206726        1.774770
LBFGS:    1 00:15:14     -759.414429        0.615212
LBFGS:    2 00:15:17     -759.489624        0.512618
LBFGS:    3 00:15:19     -759.557556        0.337190
LBFGS:    4 00:15:23     -759.597656        0.271241
LBFGS:    5 00:15:26     -759.621399        0.221883
LBFGS:    6 00:15:28     -759.636230        0.191616
LBFGS:    7 00:15:31     -759.652954        0.160077
LBFGS:    8 00:15:33     -759.666504        0.139420
LBFGS:    9 00:15:35     -759.674622        0.140361
LBFGS:   10 00:15:38     -759.679443        0.134428
LBFGS:   11 00:15:40     -759.684509        0.114731
LBF

Now we need to create two new supercells (min_cell_1 and min_cell_2) with a Li atom in the initial (pos_1) and final (pos_2) positions that we just identified. We do this by adding Li back into the structure we just created containing two Li vacancies. Once we have added a Li to the end of the file ([-1]), we optimise the atomic positions in the two new supercells. 

## Step 5: Set up dimer calculation
We now have our optimised minimum structure for our dimer calculation. Assuming that we have produced a trajectory (.traj) file during the optimisation, we can read this in to give us our initial structure for the dimer (we can also directly use min_cell_1 if we want to run everything in one go).

In [7]:
dimer_cell=read('min_cell_1_opt.traj')
dimer_cell.calc=mace_mp(default_dtype="float32") #use float64 for higher precision
traj = Trajectory('dimer.traj', 'w', dimer_cell)
traj.write()

Using Materials Project MACE for MACECalculator with /Users/ieuanseymour/.cache/mace/5yyxdm76
Using float32 for MACECalculator, which is faster but less accurate. Recommended for MD. Use float64 for geometry optimization.
Default dtype float32 does not match model dtype float64, converting models to float32.


We now want to specify which atoms to initially displace in the dimer calculation with d_mask. We only want to displace the final Li atom in the cell, so we set the mask for this atom as True and the rest as False.

In [8]:
N = len(dimer_cell)
d_mask = [False] * (N - 1) + [True]
d_control = DimerControl(initial_eigenmode_method='displacement',displacement_method='vector',logfile=None,mask=d_mask)
d_atoms = MinModeAtoms(dimer_cell, d_control)

Next, we need to specify the displacement vector on the atoms in our structure. We only want to displace the last Li in the structure a small amount along the b-axis. We start by creating a 3N vector of zeros (0, 0, 0) for all the atoms in the structure. We then modify the last entry to give a displacement of 0.3A along the b direction (0, 0.3, 0)

In [9]:
displacement_vector = np.zeros((N, 3))

In [10]:
################## User input #################
displacement_vector[-1] = [0, 0.3, 0]
################## User input #################

In [11]:
d_atoms.displace(displacement_vector=displacement_vector)

## Step 6: Run dimer calculation and analyze results
It's now time to run our dimer! Just as we did for the geometry optimisation, we assign an optimiser and 'run'. The path of the dimer run as it walks up hill is saved as 'dimer.traj' which we can visualise in ase gui. 

In [12]:
dim_rlx = MinModeTranslate(d_atoms,trajectory=traj,logfile=None)
dim_rlx.run(fmax=0.05)
diff = dimer_cell.get_potential_energy() - e0
print('The energy barrier is %f eV.' % diff)

write('transition_state.cif',dimer_cell,format='cif')

view(dimer_cell, viewer='x3d')

fname='dimer.traj@:'
path_total=read(fname)
view(path_total)


The energy barrier is 0.230164 eV.


<Popen: returncode: None args: ['/Library/Frameworks/Python.framework/Versio...>